<a href="https://colab.research.google.com/github/kaggler-KyotoUni/kaggle-titanic/blob/chopprin_branch/pandas_excercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

In [4]:
%load_ext autoreload
import pandas as pd 
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import numpy as np 

import matplotlib.pyplot as plt 
plt.rc('figure',figsize=[10.0, 3.0])

import seaborn as sns; sns.set()
import os
import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from itertools import cycle
pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

# メモリ解放の為
import gc

from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf 
tf.__version__

# データは1回しか読み込みしなくてもgoogle colab上に移したほうが速いとの事
# !mkdir "/content/data/"
# !cp -r "/content/drive/My Drive/kaggle/data/titanic" "/content/data/" 
# moduleはgoogle driveにおいとく
# !mkdir "/content/module/"
# !cp -r "/content/drive/My Drive/kaggle/module" "/content/module/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


'2.2.0'

In [5]:
# 自作もジュールのimport
%autoreload
%cd "/content/drive/My Drive/kaggle/module/"
import load_data as ld
import eda_data as ed

/content/drive/My Drive/kaggle/module


In [6]:
%cd /content/
!ls
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
gender = pd.read_csv("./gender_submission.csv")

print("Read finished")
print("@@@@@@@@@@@@@@@@@@")

csv_list = [train, test]
#csv_list = [cal, stv, ste, ss, sellp]

for i, csv in enumerate(tqdm(csv_list)):
    csv_list[i] = ld.reduce_mem_usage(csv)
print("Memory usage reduced")
print("@@@@@@@@@@@@@@@@@@")

/content
adc.json  drive  gender_submission.csv	sample_data  test.csv  train.csv


100%|██████████| 2/2 [00:00<00:00, 87.29it/s]

Read finished
@@@@@@@@@@@@@@@@@@
Mem. usage decreased to  0.04 Mb (47.8% reduction)
Mem. usage decreased to  0.02 Mb (44.2% reduction)
Memory usage reduced
@@@@@@@@@@@@@@@@@@


In [7]:
for i, csv in enumerate(tqdm(csv_list)):
    csv_list[i] = csv.fillna(0)

100%|██████████| 2/2 [00:00<00:00, 120.43it/s]


# ApplyとMap関数について
- kaggleのtitanicのデータを用意すること

In [69]:
print("train shape : {}".format(train.shape))
print("test shape : {}".format(test.shape))

train shape : (891, 12)
test shape : (418, 11)


apply：DataFrame・Seriesの行・要素に関数を適用


In [67]:
f_add_some_value = lambda x: x + 10
train["Age"].apply(f_add_some_value)

0      32.0
1      48.0
2      36.0
3      45.0
4      45.0
       ... 
886    37.0
887    29.0
888     NaN
889    36.0
890    42.0
Name: Age, Length: 891, dtype: float64

In [64]:
# 引数が２つ以上の場合もapplyで対応可能
def f_add_some_value(column, value):
  return column + value

# 1つめの引数はtrain["Age"]に含まれており，2つめ以降の引数をapply中で適用
train["Age"].apply(f_add_some_value, value = 10)

0      32.0
1      48.0
2      36.0
3      45.0
4      45.0
       ... 
886    37.0
887    29.0
888     NaN
889    36.0
890    42.0
Name: Age, Length: 891, dtype: float64

In [51]:
# value_counts()はSeriesに対してのみ適用可能
index = 1
train.iloc[:,index].value_counts()

# 無理矢理DataFrameに対してvalue_countsを適用する場合はapplyを使えば可能
f_value_counts = lambda x: x.value_counts()
train.apply(f_value_counts)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,NaN,549.0,NaN,NaN,NaN,NaN,608.0,678.0,NaN,15.0,NaN,NaN
0.419921875,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
0.669921875,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
0.75,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
0.830078125,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
"del Carlo, Mr. Sebastiano",NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
female,NaN,NaN,NaN,NaN,314.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
male,NaN,NaN,NaN,NaN,577.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"van Billiard, Mr. Austin Blyler",NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Series各要素に関数を適用：map

In [31]:
index = 3
f_brackets = lambda x: '[{}]'.format(x) # 自作関数の定義
print(train.iloc[:,3].map(f_brackets))

0                              [Braund, Mr. Owen Harris]
1      [Cumings, Mrs. John Bradley (Florence Briggs T...
2                               [Heikkinen, Miss. Laina]
3         [Futrelle, Mrs. Jacques Heath (Lily May Peel)]
4                             [Allen, Mr. William Henry]
                             ...                        
886                              [Montvila, Rev. Juozas]
887                       [Graham, Miss. Margaret Edith]
888           [Johnston, Miss. Catherine Helen "Carrie"]
889                              [Behr, Mr. Karl Howell]
890                                [Dooley, Mr. Patrick]
Name: Name, Length: 891, dtype: object
